# Processamento dos dados

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import DataFrame

In [0]:
df_offers = spark.read.json('/Volumes/workspace/raw/dados_brutos/offers.json')
df_profile = spark.read.json('/Volumes/workspace/raw/dados_brutos/profile.json')
df_trans = spark.read.json('/Volumes/workspace/raw/dados_brutos/transactions.json')

## 

## Manipulação e limpeza dos dados

In [0]:
def verificar_nulos(df: DataFrame) -> DataFrame:

    """
    Função para verificar a quantidade de dados nulos por coluna. 
    Parâmetros:
        df: Spark DataFrame
    """

    df_nulos = df.select([
        F.sum(F.col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ])

    return df_nulos


def verificar_duplicatas(df: DataFrame, coluna: str = 'id') -> DataFrame:

    """
    Função para verificar a quantidade de duplicatas assumindo a granularidade id
    Parâmetros:
        df: Spark DataFrame
    """

    df_duplicatas = df.groupBy(coluna).count().filter(F.col('count') > 1).count()

    return df_duplicatas

### 1 - Offers

In [0]:
df_offers.limit(5).display()

In [0]:
df_offers.printSchema()

In [0]:
print(f"Quantidade de duplicatas: {verificar_duplicatas(df_offers)}")

In [0]:
verificar_nulos(df_offers).display()

### 2 - Profile

In [0]:
df_profile.limit(5).display()

In [0]:
df_profile.printSchema()

In [0]:
verificar_nulos(df_profile).display()

Vamos verificar se os indices dos valores presentes em `credit_card_limit` são os mesmos de `gender`. Olhando as 5 primeiras linhas, parece que há um padrão que toda vez que a idade é 118, vamos verificar isso. 

In [0]:
qtd_dados_nulos = df_profile.filter(
    (F.col("age") == 118) & 
    (F.col("gender").isNull()) & 
    (F.col("credit_card_limit").isNull())
).count()

print(f" Quantidade de de dados nulos para idade 118 e nulos para gender e credit_card_limit: {qtd_dados_nulos}")

Confirmada a suspeita de que todos os dados que aparecem nulo `credit_card_limit` e `gender` possui idade 118. 

In [0]:

print(f"Percentual da base de usuários que é nula: {qtd_dados_nulos/df_profile.count()*100:.2f}%")

In [0]:
gender_null = F.col("gender").isNull()
limit_null  = F.col("credit_card_limit").isNull()

nulo_ambos = df_profile.filter(gender_null & limit_null).count()

print(f'Quantidade de linhas com ambos os campos nulos: {nulo_ambos}')

Portando todos os dados que estão nulos em `gender` também estão nulos em `credit_card_limit`


In [0]:
df_profile = df_profile.withColumn("registered_on", F.to_date(F.col("registered_on"), "yyyyMMdd"))

In [0]:
print(f"Quantidade de duplicatas: {verificar_duplicatas(df_profile)}")

### 3 - Transactions

In [0]:
df_trans.limit(5).display()

In [0]:
df_trans.printSchema()

In [0]:
colunas_selecionadas = ['account_id', 'event', 'time_since_test_start']

df_trans = df_trans.select(
    *colunas_selecionadas,
    "value.*"
)

In [0]:
verificar_nulos(df_trans).display()

Temos duas colunas que parecem carregar o mesmo significado, sendo elas `offer id` e `offer_id`, porém percebe-se que a quantidade de dados nulos em cada uma diferente. Vamos verificar essas colunas com mais calma. 


In [0]:
from pyspark.sql import functions as F

eh_offerid_nulo = F.col("offer id").isNull()
eh_offer_id_nulo = F.col("offer_id").isNull()

resumo = df_trans.agg(
    F.count('*').alias('total_linhas'),
    F.sum(F.when(eh_offerid_nulo & eh_offer_id_nulo, 1).otherwise(0)).alias('ambos_nulos'),
    F.sum(F.when(~eh_offerid_nulo & eh_offer_id_nulo, 1).otherwise(0)).alias('apenas_underscore_nulo'),
    F.sum(F.when(eh_offerid_nulo & ~eh_offer_id_nulo, 1).otherwise(0)).alias('apenas_espaco_nulo'),
    F.sum(F.when(~eh_offerid_nulo & ~eh_offer_id_nulo, 1).otherwise(0)).alias('ambos_nao_nulos')
    

).first()

total_linhas = resumo['total_linhas']
ambos_nulos = resumo['ambos_nulos']
apenas_underscore_nulo = resumo['apenas_underscore_nulo']
apenas_espaco_nulo = resumo['apenas_espaco_nulo']
ambos_n_nulos = resumo['ambos_nao_nulos']

if total_linhas > 0:
    print(
        f"Ambos nulos: {ambos_nulos} / {total_linhas} = {ambos_nulos/total_linhas*100:.2f}%"
    )
    print(
        f"Apenas 'offer id' nulo: {apenas_espaco_nulo} / {total_linhas} = {apenas_espaco_nulo/total_linhas*100:.2f}%"
    )
    print(
        f"Apenas 'offer_id' nulo: {apenas_underscore_nulo} / {total_linhas} = {apenas_underscore_nulo/total_linhas*100:.2f}%"
    )

    print(
        f"Ambos não nulos: {ambos_n_nulos} / {total_linhas} = {ambos_n_nulos/total_linhas*100:.2f}%"
    )
else:
    print("O DataFrame está vazio.")


Observa-se que quase metade dos registros não tem nenhuma informação de oferta associada, esses registros provavelmente são referentes a **eventos de transação normal**, ou seja, compras feitas sem cupons ou ofertas. 

Era esperado que as colunas `offer id` e `offer_id` fossem duplicadas, mas a análise mostra que não são. 

* Há casos em que só `offer id` está preenchido (~11%)

* E casos em que só `offer_id` está preenchido (~44%)

Indicando que os dados de oferta estão espalhados entre as duas colunas. 

Outra informação que obtemos é que ambos não nulo é 0%, indicando que nunca houve uma linha com os dois preenchidos ao mesmo tempo, uma ou a outra é usada, mas nunca as duas. Vamos unificar essas colunas posteriormente.

Outro detalhe que percebemos é que `reward` possui a mesma quantidade de dados nulos que a coluna `offer_id`, indicando que essas duas colunas possuem certa relação. Vamos analisar essa relação. 



In [0]:
df_trans.select(
    F.when(F.col("offer id").isNotNull(), "tem_offer_id_espaco")
     .when(F.col("offer_id").isNotNull(), "tem_offer_id_underscore")
     .otherwise("sem_offer").alias("origem_offer"),
    F.when(F.col("reward").isNotNull(), 1).otherwise(0).alias("tem_reward")
).groupBy("origem_offer").agg(
    F.count("*").alias("total"),
    F.sum("tem_reward").alias("com_reward")
).display()


1. **`tem_offer_id_underscore` (33.579 eventos)**

   * Todos esses eventos têm **`offer_id`**.
   * E **todos eles têm `reward` associado** (33.579/33.579).
     Isso significa que **nessa versão do schema, o campo `reward` foi registrado corretamente**.

2. **`tem_offer_id_espaco` (134.002 eventos)**

   * Todos têm uma oferta vinculada (`offer id`).
   * Mas **nenhum deles tem `reward` preenchido**.
     Isso indica que quando a ingestão usou a chave `offer id`, a informação de recompensa não veio junto. **Ou seja, há uma perda de informação.**

3. **`sem_offer` (138.953 eventos)**

   * Eventos que não têm nenhuma oferta associada.
   * Naturalmente não têm `reward`.
     Faz sentido, pois são **transações normais**, sem cupom.

Isso indica que o campo `reward` só é confiável para as ofertas registras em `offer_id`. As ofertas que vieram de `offer id` ficaram sem `reward`, mesmo quando provavelmente havia uma recompensa. Isso indica uma inconsistência nos dados. Aparentemente duas versões do mesmo campo foram mescladas, mas só uma preservou o reward. 

Como reward é o valor do desconto recebido, uma abordagem que podemos seguir pegar esse valor de reward diretamente da tabela *offers*.  Lá cada Offer tem um campo de desconto, dessa forma conseguimos recuperar de lá. Faremos isso posteriormente na etapa de unificação das bases. 

In [0]:
df_trans_final = df_trans.withColumn(
    "offer_id_final",
    F.coalesce(F.col("offer_id"), F.col("offer id"))
).drop("offer_id", "offer id")

Vamos verificar quais são as ofertas que aparecem em maior quantidade. 

In [0]:
df_duplicatas = df_trans_final.groupBy("offer_id_final").count()
display(df_duplicatas)

Databricks visualization. Run in Databricks to view.

In [0]:
df_trans_final.write.mode("overwrite").saveAsTable("workspace.processed.transactions_processed")

## Preparação de dataset unificado

In [0]:
df_join = (
    df_trans_final.alias("t")
    .join(df_profile.alias("p"), F.col("t.account_id") == F.col("p.id"), "inner")
    .join(df_offers.alias("o"), F.col("t.offer_id_final") == F.col("o.id"), "left")
    .drop("id")
)


In [0]:
df_join.display()

In [0]:
print(f"Quantidade de linhas antes de join: {df_trans_final.count()}")
print(f"Quantidade de linhas depois de join: {df_join.count()}")

Ou seja não houve perda de dados

In [0]:
df_join.filter(
    (F.col("reward").isNotNull()) & (F.col("reward") != F.col("discount_value"))
).display()


Como mencionado acima, o valor de `reward` é valor do disconto, portando podemos deixar somente a coluna `discount_value`

In [0]:
df_join = df_join.drop('reward')

In [0]:
df_join.groupBy("time_since_test_start").count().display()